# Client Retention Demo Using Python
In this demo, we will show Anaconda functionality accessing enterprise data from VSAM and DB2. The data stored in VSAM consists of 6,001 rows of customer information.  The data stored in DB2 consists of 20,000 rows of transaction data. The data is transformed and joined in a Pandas DataFrame, which is used to perform exploratory analyses. A random forest algorithm is then used to predict customer churn.

In [ ]:
USERNAME="???????"
PASSWORD="????????"
MDSS_SSID="AZK1"
DB2_SSID="DBBG"

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore", category=PendingDeprecationWarning)

## Set up Mainframe Data Connections
This step will set up the VSAM and DB2 connections to access the data and load them into Pandas DataFrames.  The dsdbc module is delivered with the z/OS IzODA Anaconda distribution. It enables Python applications to access the z/OS IzODA Mainframe Data Service. The Data Service component (MDS) provides optimized, virtualized, and parallelized access to both IBM Z data sources and other off-platform data sources.

In [ ]:
def cp1047_to_utf8(list):
    list_out = []
    for e in list:
        x = ()
        for i in e:
            if isinstance(i, (str,)):
                s = i.encode('utf16').decode('cp1047').encode('utf8').decode('utf16')[2:]
                x = x + (s,)
            else:
                x = x + (i,)
        list_out.append(x)
    return list_out

def load_data_from_mds(vtable_name, user, password, mds_id=MDSS_SSID):
    import dsdbc
    conn =dsdbc.connect(SSID=mds_id, user=user, password=password)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM " + vtable_name)
    rows = cursor.fetchall()
    label = []
    for col in cursor.description: label.append(col[0].lower())
    conn.close()
    return pd.DataFrame(rows, columns=label)

def load_data_from_db2(table_name, user, password, mds_id=MDSS_SSID,  db2_id=DB2_SSID):
    import dsdbc
    conn =dsdbc.connect(SSID=mds_id, user=user, password=password, dsid=db2_id)
    cursor = conn.cursor()
    sql = "SELECT * FROM " + table_name
    #print(sql)
    cursor.execute(sql)
    rows = cp1047_to_utf8(cursor.fetchall())
    label = []
    for col in cursor.description: label.append(col[0].lower())
    conn.close()
    return pd.DataFrame(rows, columns=label)


***Credit card transactions***

Load credit card transactions into a Pandas DataFrame.

In [ ]:
txn_df = load_data_from_db2(table_name='SPARKDB.SPPAYTB1', user=USERNAME, password=PASSWORD)

In [ ]:
txn_df['acaureq_aureq_tx_dt_ttlamt'] = pd.to_numeric(txn_df['acaureq_aureq_tx_dt_ttlamt'])
txn_df['cont_id'] = txn_df['cont_id'].astype('int64')
txn_df['acaureq_hdr_credtt'] = pd.to_datetime(txn_df['acaureq_hdr_credtt'])

In [ ]:
txn_df['date'] = txn_df['acaureq_hdr_credtt'].apply(lambda x: x.date())
txn_df

***Client Data***

Load client data into a Pandas DataFrame.

In [ ]:
client_df = load_data_from_mds(vtable_name='VSAM_CLIENT', user=USERNAME, password=PASSWORD)
client_df = client_df.set_index("cont_id")
client_df

## Aggregate statistics
Calculate a few aggregate statistics based on credit transactions and join the results to the client data DataFrame.

In [ ]:
# Total transactions per customer
total_txns_df = txn_df.groupby('cont_id').size().rename("total_txns").to_frame()
client_df = total_txns_df.join(client_df)

In [ ]:
# Total transaction amounts per customer
total_txn_amount_df = txn_df.groupby('cont_id')['acaureq_aureq_tx_dt_ttlamt'].sum().rename("total_txn_amount").to_frame()
client_df = client_df.join(total_txn_amount_df)

In [ ]:
# Average transaction amounts per customer
avg_txn_amount_df = txn_df.groupby('cont_id')['acaureq_aureq_tx_dt_ttlamt'].mean().rename("avg_txn_amount").to_frame()
client_df = client_df.join(avg_txn_amount_df)

In [ ]:
# Average daily transactions per customer
daily_txns = txn_df.groupby(['date', 'cont_id']).size()

In [ ]:
# Missing transactions on a particular day means customer had none.
# These days should be included in the average as 0 transaction days.
avg_daily_txns_df = daily_txns.unstack().fillna(0).mean().rename("avg_daily_txns").to_frame()
client_df = client_df.join(avg_daily_txns_df)
client_df

### Exploratory Analyses
We begin our exploration of the data set by creating a scatterplot of annual_income vs. age_years and their associated histograms. Matplotlib and Seaborn are two common plotting libraries used in Python.  These plotting libraries are useful in creating custom visualizations to help gain insights from our data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
%matplotlib inline

In [ ]:
def jointplot(x, y, data, **kwargs):
    size = kwargs.pop('size', 10)
    alpha = kwargs.pop('alpha', 0.3)
    return sns.jointplot(x=x, y=y, data=data, 
                         alpha=alpha,
                         size=size,
                         **kwargs)

# for widget
def w_jointplot(x, y):
    g = jointplot(x, y, filter_outliers(client_df, by_col=y))
    plt.close()
    return g.fig

In [ ]:
churn_labels = ['Did not churn', 'Did churn']

def filter_outliers(d, by_col=None):
    if isinstance(d, pd.
                  Series):
        return d[((d-d.mean()).abs()<=3*d.std())]
    elif isinstance(d, pd.DataFrame):
        if not by_col:
            raise ValueError('by_col is required for DataFrame')
        return d[np.abs(d[by_col]-d[by_col].mean())<=(3*d[by_col].std())] 

In [ ]:
ax = jointplot('age_years', 'annual_income', filter_outliers(client_df, by_col='annual_income'))

### Correlations
Next, we compute the correlation coefficients between each variable and create a color-coded correlation matrix. 

In [ ]:
corr = client_df.corr()

# only show lower triangle
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(12,12))
ax = sns.heatmap(corr, mask=mask, square=True, annot=True, fmt='.2f',
                 cbar=True,
                 ax=ax)
title = ax.set_title('Correlations', size=14)

## Churn
Here we plot the distributions of clients who did and did not churn. The green histogram shows the number of clients who did churn. The blue histogram shows the number of clients who did not churn.  The line graphs show the density functions for each case. 

In [ ]:
def plot_churn_by(df, col, **kwargs):
    f, ax = plt.subplots(figsize=(12,10), sharex=True)
    kde = kwargs.get('kde', False)
    hist = kwargs.get('hist', False)
    for churn in df.churn.unique():
        sns.distplot(df[df.churn == churn][col], 
                     label=churn_labels[churn], 
                     kde_kws={'shade': (kde and not hist)},
                     ax=ax, 
                     **kwargs)

    ax.set_title('Client Churn by {}'.format(col))
    label = ax.set_xlabel('{}'.format(col))
    return f, ax

def w_plot_churn_by(column, hist=True, kde=False, norm_hist=False):
    df = filter_outliers(client_df, by_col=column)
    f, ax = plot_churn_by(df, column, hist=hist, kde=kde, norm_hist=norm_hist)
    plt.legend()
    plt.close()
    return f

f, ax = plot_churn_by(client_df, 'age_years')
ax = plt.legend()

As shown in the correlation matrix above, the two features that showed a negative correlation with churn were age and activity level. Here we generate a boxplot with those two features as the axes, and churn as the category. The plot shows that clients that churn tend to be younger across all levels of activity.

In [ ]:
col = 'age_years'
data = filter_outliers(client_df, by_col=col)

f, ax = plt.subplots(figsize=(12,8))
ax = sns.boxplot(x='activity_level', y=col, hue="churn", data=data, 
                 palette='muted', ax=ax)
title = ax.set_title('Client Churn by Activity Level')
label = ax.set_ylabel('Age (Years)')
label = ax.set_xlabel('Activity Level')
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles, churn_labels)

This beeswarm plot shows clients binned by the level of activity they maintain with the bank. Clients that churned maintained lower levels of activity (0-2). And of clients within these lower activity levels, younger clients churned more than others.

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
ax = sns.swarmplot(x='activity_level', y='age_years', hue='churn', 
                   data=data.sample(n=100, random_state=51), 
                   palette='muted', ax=ax)
title = ax.set_title('Client Churn by Activity Level')
label = ax.set_ylabel('Age (Years)')
label = ax.set_xlabel('Activity Level')
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles, churn_labels)

## Train churn model
We now start to do some predictive analyses on the data to evaluate customer churn. To keep things simple, we use a single data set, which we split into training and test data sets. We use the training data to train the model, and the test data to make predictions about lost revenue to the bank.

We use a supervised learning algorithm, random forest, to train the model. Random Forest is a popular algorithm for both classification and regression. It requires very little tuning and is less prone to overfitting. Random forest is an aggregation of decision trees where each tree classifies an observation in a dataset. Since random forest aggregates many classifiers, it is considered an ensemble method. Using scikit learn, we create our random forest for classification. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
def make_feature_space(df):
    '''Create the feature space required by our classifier.'''
    # drop columns/features we don't want/need for the classifier
    features_df = df.drop(['churn', 'customer_id'], axis=1, errors='ignore')
    X = features_df.as_matrix().astype(np.float)
    # normalize feature values
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X

def predict_churn(X):
    '''Predict the probabilit of churn from feature set.'''
    return clf.predict_proba(X)[:,1]

def train_model(X, y):
    '''Train our classifier using features X and target variable y.'''
    clf = RF(n_estimators=100)
    return clf.fit(X, y)

def init_model(df):
    # split data into train, test sets
    train_index, test_index = train_test_split(df.index, random_state=99)
    train_df = client_df.ix[train_index]
    test_df = client_df.ix[test_index]

    # target variable
    y = np.array(train_df['churn'])

    # extract features
    X = make_feature_space(train_df)

    # train classifier
    clf = train_model(X, y)

    return clf, test_df

After training the model, the churn classifier and the test data set are used for our churn predictions.

In [ ]:
clf, test_df = init_model(client_df)

## Calculate business loss
In this simple example, we calculate the predicted loss of business (revenue) for all clients in the test data set. We calculate the revenue from each client, and multiply that by the churn probability to determine the predicted loss.

In [ ]:
def calc_business_loss(df):
    df['customer_id'] = df.index
    data = df.copy()

    # extract features
    X = make_feature_space(df)
    
    # predict churn
    data['churn_probability'] = predict_churn(X)
    
    # TODO: avg_daily_balance would be a nice feature to have here
    # for now, we'll just use fraction of income
    avg_daily_balance = df['annual_income'] / 6

    # Interest made on deposits
    deposit_rate = 0.02

    # Fee collected for each credit txn
    credit_rate = 0.015

    # Assume we make some money on trading fees and/or portfolio management
    mgmt_rate = 0.02

    # How much is each customer worth to the business?
    worth = deposit_rate * avg_daily_balance + \
            mgmt_rate * df['annual_invest'] + \
            credit_rate * df['total_txn_amount']
    data['worth'] = worth
    
    # How much would we lose per annum?
    data['predicted_loss'] = data['churn_probability'] * worth
    
    return data.sort_values(by='predicted_loss', ascending=False)

In [ ]:
churn_df = calc_business_loss(test_df)
churn_df.head()

## Loss by Age Group
In this section, we calculate and plot the predicted loss of revenue by age group. In our data set, age is an important feature in predicting if a client will churn. We create a DataFrame containing the cumulative predicted loss by age group.

In [ ]:
def group_by_age(df, bins=None):
    if bins is None:
        bin_size = 5
        _min, _max = int(df.age_years.min()), int(df.age_years.max())
        bins = range(_min, _max + bin_size, 5)
    return df.groupby(pd.cut(df.age_years, bins=bins))

data_by_age = churn_df.pipe(group_by_age)

In [ ]:
loss_by_age_df = data_by_age['predicted_loss'].sum().reset_index()
loss_by_age_df['age_years'] = loss_by_age_df['age_years'].astype(str)

loss_by_age_df.plot(x='age_years', y='predicted_loss', style='o')